In [ ]:
##### SPLIT THEM INTO 22 GROUPS 
import os
import numpy as np
import pandas as pd

# CONFIG

INPUT_FILE = "nuclei_all.xlsx"
OUTPUT_DIR = "splits"
OUTPUT_FILE = f"{OUTPUT_DIR}/random_22_groups.xlsx"

N_GROUPS = 22
SEED = 42   # for reproducibility

# LOAD DATA

df = pd.read_excel(INPUT_FILE)[["Z", "N", "A"]].copy()

# Apply A < 121 cut
df = df[df["A"] > 0].reset_index(drop=True)

# RANDOM SHUFFLE

rng = np.random.default_rng(SEED)
perm = rng.permutation(len(df))
df = df.loc[perm].reset_index(drop=True)

# ASSIGN GROUPS EVENLY

group_ids = np.zeros(len(df), dtype=int)

sizes = [len(df) // N_GROUPS] * N_GROUPS
for i in range(len(df) % N_GROUPS):
    sizes[i] += 1

start = 0
for g, sz in enumerate(sizes, start=1):
    group_ids[start:start+sz] = g
    start += sz

df["group"] = group_ids

# SAVE

os.makedirs(OUTPUT_DIR, exist_ok=True)
df[["Z", "N", "group"]].to_excel(OUTPUT_FILE, index=False)

# PRINT OUTPUT SUMMARY

print("\nGroup sizes:")
print(df["group"].value_counts().sort_index())

print("\nSaved splits to:", OUTPUT_FILE)
print("===================================================")


In [ ]:
#TRAIN OVER ALL GROUPS

import sys

print("Notebook Python:", sys.executable)

for i in range(1, 23):
    print("\n" + "="*60)
    print(f"RUNNING GROUP {i}")
    print("="*60)
    
    !{sys.executable} STEP_1_Feed_Forward_Neural_Nework.py {i}

In [ ]:
#To plot all the nuclei 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.stats import linregress
import os

# LOAD ALL GROUP FILES

all_dfs = []

for i in range(1, 23):
    file_path = f"results_residual_ffnn_singlebin/Test_Group_{i}_Final.xlsx"
    
    if os.path.exists(file_path):
        df_i = pd.read_excel(file_path)
        all_dfs.append(df_i)

df_all = pd.concat(all_dfs, ignore_index=True)

print("Total nuclei:", len(df_all))

# BOUSTROPHEDON (|N-Z| BASED)

def boustrophedon_sort(df):

    df = df.copy()
    df["abs_N_minus_Z"] = np.abs(df["N"] - df["Z"])

    ordered_blocks = []

    for A_val in sorted(df["A"].unique()):

        df_A = df[df["A"] == A_val]

        if A_val % 2 == 0:
            df_A = df_A.sort_values("abs_N_minus_Z", ascending=True)
        else:
            df_A = df_A.sort_values("abs_N_minus_Z", ascending=False)

        ordered_blocks.append(df_A)

    df_final = pd.concat(ordered_blocks, ignore_index=True)
    df_final.drop(columns=["abs_N_minus_Z"], inplace=True)

    return df_final

df_bous = boustrophedon_sort(df_all)

delta = df_bous["deltaMi_final"].values
N = len(delta)

# FIGURE 1

plt.figure(figsize=(14,6))
plt.plot(delta)
plt.title("Figure 1: ΔM_final (Boustrophedon)")
plt.xlabel("j")
plt.ylabel("ΔM_final (MeV)")
plt.grid(True)
plt.tight_layout()
plt.show()

# ============================================================
# FIGURE 2
# ============================================================

delta_centered = delta - np.mean(delta)
fft_vals = fft(delta_centered)
power = np.abs(fft_vals)**2
freqs = np.fft.fftfreq(N)

mask = freqs > 0
freqs = freqs[mask]
power = power[mask]

fit_mask = freqs > 0.01

log_f = np.log10(freqs[fit_mask])
log_p = np.log10(power[fit_mask])

slope, intercept, r_value, _, _ = linregress(log_f, log_p)

fit_line = 10**(intercept + slope * log_f)

plt.figure(figsize=(14,6))
plt.loglog(freqs, power)
plt.loglog(freqs[fit_mask], fit_line, 'r--',
           label=f"Slope = {slope:.3f}")
plt.title("Figure 2: Fourier Power Spectrum")
plt.xlabel("Frequency")
plt.ylabel("Power")
plt.grid(True, which="both")
plt.legend()
plt.tight_layout()
plt.show()

print(f"\nSpectral slope = {slope:.4f}")
print(f"R^2 = {r_value**2:.4f}")